In [1]:
import tensorflow as tf
from connectionist.layers import MultiInputTimeAveraging

print(MultiInputTimeAveraging.__doc__)

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Time-averaging mechanism for multiple inputs.

    In short, time-averaging mechanism simulates continuous-temporal dynamics in a discrete-time recurrent neural networks.
    See Plaut, McClelland, Seidenberg, and Patterson (1996) equation (15) for more details.

    This layer is designed for multiple inputs, assuming they had ALREADY been multiplied by weights i.e., a list of (x @ w).

    Args:
        tau (float): Time-averaging parameter (How much information should take from the new input). range: [0, 1].

        average_at (str): Where to average. Options: 'before_activation', 'after_activation'.

            When average_at is 'before_activation', the time-averaging is applied BEFORE activation. i.e., time-averaging INPUT.:
                outputs = activation(integrated_input);
                integrated input = tau * (sum(inputs) + bias) + (1-tau) * last_inputs;
                last_inputs is obtained from the last call of this layer, its values stored at `self.states`

    

## Toy example

This example illustrate the forward pass of a model bullt with the multi-input time-averaging model.

$$y_t = \tau \times sigmoid(x_1 @ w_1 + x_2 @ w_2 + b) + (1-\tau) y_{t-1}  $$

In [2]:
class ToyModel(tf.keras.layers.Layer):

    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, input_shape):
        self.dense_1 = tf.keras.layers.Dense(units=self.units, use_bias=False)
        self.dense_2 = tf.keras.layers.Dense(units=self.units, use_bias=False)
        self.mita = MultiInputTimeAveraging(tau=0.2, average_at="after_activation", activation="sigmoid")
        self.built = True

    def call(self, inputs):
        x1, x2 = inputs
        y1 = self.dense_1(x1)
        y2 = self.dense_2(x2)
        y = self.mita([y1, y2])
        return y

model = ToyModel(units=3)


First call, since tau = 0.2, output values $y_1 = 0.2 \times sigmoid(x_1 @ w_1 + x_2 @ w_2 + b)$

In [5]:
y1 = model([tf.constant([[1, 2, 3, 4]]), tf.constant([[1, 2, 3, 4, 5]])])
y1

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.48508698, 0.01522337, 0.43136233]], dtype=float32)>

Second call, output ramping up, output value $y_2 = 0.2 \times sigmoid(x_1 @ w_1 + x_2 @ w_2 + b) + 0.8 y_1$.

In [4]:
y2 = model([tf.constant([[1, 2, 3, 4]]), tf.constant([[1, 2, 3, 4, 5]])])

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.35785103, 0.01123036, 0.3182181 ]], dtype=float32)>